In [3]:
import pandas as pd
import numpy as np
import itertools
import arrow
import datetime
import time
import aiohttp
import asyncio

In [2]:
tickers = ['BERGEPAINT','BHARTIARTL','DRREDDY']

In [29]:
async def async_quote_data(symb):
    client = aiohttp.ClientSession()
    try:
        data_range = "30m"
        data_interval = "5m"
        symbol = symb + '.NS'
        res = await client.get('https://query1.finance.yahoo.com/v8/finance/chart/{symbol}?range={data_range}&interval={data_interval}'.format(**locals()))
        data = await res.json()
        body = data['chart']['result'][0]    
        dt = datetime.datetime
        dt = pd.Series(map(lambda x: arrow.get(x).to('Asia/Calcutta').datetime.replace(tzinfo=None), body['timestamp']), name='Datetime')
        df = pd.DataFrame(body['indicators']['quote'][0], index=dt)
        dg = pd.DataFrame(body['timestamp'])    
        df = df.loc[:, ('open', 'high', 'low', 'close', 'volume')]
        df.dropna(inplace=True)     #removing NaN rows
        df.columns = ['Open', 'High','Low','Close','Volume']    #Renaming columns in pandas
        df.index.name = 'Date'
        await client.close()
        return df
    except Exception as e:
        await client.close()
        print("Error at: ", symb," : ",str(e))

In [11]:
for i in range(3):
    await asyncio.gather(*[async_quote_data(ticker) for ticker in tickers])
    time.sleep(1)

DRREDDY                             Open         High          Low        Close  \
Date                                                                      
2020-11-11 11:35:00  4747.450195  4753.850098  4744.000000  4751.000000   
2020-11-11 11:40:00  4749.149902  4767.000000  4749.149902  4765.750000   
2020-11-11 11:45:00  4765.700195  4787.049805  4764.049805  4781.100098   
2020-11-11 11:50:00  4780.149902  4803.799805  4780.149902  4801.600098   
2020-11-11 11:55:00  4801.549805  4808.000000  4775.000000  4782.250000   
2020-11-11 12:00:00  4779.250000  4787.600098  4779.250000  4786.450195   
2020-11-11 12:03:32  4790.000000  4790.000000  4790.000000  4790.000000   

                     Volume  
Date                         
2020-11-11 11:35:00    7206  
2020-11-11 11:40:00   19593  
2020-11-11 11:45:00   40630  
2020-11-11 11:50:00   50392  
2020-11-11 11:55:00   42591  
2020-11-11 12:00:00   10969  
2020-11-11 12:03:32       0  
BHARTIARTL                            Open    

In [35]:
dei = await async_quote_data('RELIANCE')
dei

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-11 14:25:00,2008.550049,2011.400024,2008.000000,2008.650024,34588
2020-11-11 14:30:00,2008.949951,2011.449951,2006.349976,2007.900024,163274
2020-11-11 14:35:00,2008.099976,2010.900024,2004.150024,2006.050049,241538
2020-11-11 14:40:00,2006.050049,2009.800049,2005.400024,2008.449951,144012
2020-11-11 14:45:00,2008.449951,2009.900024,2006.400024,2007.650024,124680
2020-11-11 14:50:00,2007.599976,2011.000000,2007.199951,2008.650024,131895
2020-11-11 14:55:00,2008.750000,2009.500000,2007.900024,2008.599976,21509
2020-11-11 14:56:21,2009.449951,2009.449951,2009.449951,2009.449951,0


In [38]:
!curl "https://api.kite.trade/instruments" \ - H "X-Kite-Version: 3" \ - H "Authorization: lC1LkWdvEchxkuJt1O0KhAjs4VCG84WK zh5pfekmc0zezour:vgfaU673sFRyvOsfGL0IpoUoWVhwSGKj"

curl: option -: is unknown
curl: try 'curl --help' or 'curl --manual' for more information
